In [1]:
import mrigutilities as mu
import pandas as pd
import datetime
import os,sys
import yahooquery as yq
import kite.kite_trade as zkite
import json

expiry1 = mu.last_thursday_of_month(datetime.date.today())
expiry2 = mu.last_thursday_of_month(expiry1 + datetime.timedelta(days=30))
expiry3 = mu.last_thursday_of_month(expiry2 + datetime.timedelta(days=30))
expiry1,expiry2, expiry3
session = mu.getKiteSession()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
engine = mu.sql_engine()


connection active


In [ ]:
from yahooquery import Ticker

symbol_list_yahoo = ['TATAPOWER.NS', 'TATAMOTORS.NS']

ticker = Ticker(symbol_list_yahoo) 
fin_data_dict = ticker.financial_data
fin_data_df = pd.DataFrame.from_dict(fin_data_dict, orient='index').T
fin_data_df

In [ ]:
# from xbrl import XBRLParser, GAAP, GAAPSerializer
import xml.etree.ElementTree as ET
import requests
eq_dir_qtr = os.path.join('resources','equity_research','FINRESULTS_QTRLY_AXISBANK.csv')
eq_dir_ann = os.path.join('resources','equity_research','FINRESULTS_ANNUAL_AXISBANK.csv')


def parse_xbrl(url):
    result = pd.DataFrame()
    i = 0
    s = requests.Session()
    headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"
}

    response = s.get(url,headers=headers)
    # print(response.content)
    
    # tree = ET.fromstring(response.content)
    
    # tapo = 'C:\\Users\\Santosh Bag\\Downloads\\TAPO.xml'
    # tree = ET.parse(tapo)
    # tree = ET.parse(response.text)
    # root = tree.getroot()
    root = ET.fromstring(response.text)
    results = {}
    for item in root:
        if 'in-bse-fin' in item.tag:
    #         print(item.attrib,item.text)
            results[item.tag.split('}')[1]] = item.text
    # finres = pd.DataFrame.from_dict(results, orient='index')
    # finres.rename(columns={0:i},inplace=True)
    # print(finres)
    # finres
    # result = pd.concat([result,finres],axis=1)
    # print('RESULT\n',result)
    i = i + 1
    return results



sym = os.path.basename(eq_dir).split('.')[0].split('_')[2]
# print(sym)



url = ['https://nsearchives.nseindia.com/corporate/xbrl/BANKING_104623_1105106_21042024122701.xml',
       'https://nsearchives.nseindia.com/corporate/xbrl/BANKING_101078_1023175_16012024051131.xml',
       'https://nsearchives.nseindia.com/corporate/xbrl/BANKING_97616_956099_17102023052102.xml',
       'https://nsearchives.nseindia.com/corporate/xbrl/BANKING_94218_888372_17072023062652.xml',
       'https://nsearchives.nseindia.com/corporate/xbrl/BANKING_90715_821266_17042023022426.xml']

data = pd.read_csv(eq_dir_ann)
cols = {'CONSOLIDATED / NON-CONSOLIDATED' : 'consolidated','PERIOD': 'period','PERIOD ENDED' : 'period_ended','RELATING TO' : 'related_qtr','** XBRL' : 'xbrl_url'}
data = data[cols.keys()]
data['symbol'] = sym
data.rename(columns=cols,inplace=True)
data['period_ended'] = pd.to_datetime(data['period_ended']).dt.date
data = data[data['period_ended'] > datetime.date(2020,1,1)] 
data['results'] = data['xbrl_url'].apply(lambda x: parse_xbrl(x))
print(data['results'])



In [ ]:
# Implementation of specific results file with specific name format having only one symbol

import zipfile,re
import requests
import xml.etree.ElementTree as ET


headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"
}


def parse_xbrl(url):
    result = pd.DataFrame()
    # print(url)
    i = 0
    s = requests.Session()
    response = s.get(url,headers=headers)
    # print(response.content)
    
    # tree = ET.fromstring(response.content)
    
    # tapo = 'C:\\Users\\Santosh Bag\\Downloads\\TAPO.xml'
    # tree = ET.parse(tapo)
    # tree = ET.parse(response.text)
    # root = tree.getroot()
    root = ET.fromstring(response.text)
    results = {}
    for item in root:
        if 'in-bse-fin' in item.tag:
    #         print(item.attrib,item.text)
            results[item.tag.split('}')[1]] = item.text
    finres = pd.DataFrame.from_dict(results, orient='index')
    finres.rename(columns={0:i},inplace=True)
    # print(finres)
    # finres
    # result = pd.concat([result,finres],axis=1)
    # print('RESULT\n',result)
    i = i + 1
    results = json.dumps(results)
    return results


eq_dir_qtr = os.path.join('resources','equity_research')
timestamp = datetime.datetime.now()
for r,d,f in os.walk(eq_dir_qtr):
    for file in f:
        if re.search("^FINRESULTS_.*csv",file):
        # if re.search("^FINRESULTS_ANNUAL_TATAMOTORS.*csv",file):            
            sym = os.path.basename(file).split('.')[0].split('_')[2]

            # stockbhavlist.append(os.path.join(r,file))
            file = os.path.join(r,file)
            print(file)
            data = pd.read_csv(file)
            cols = {'CONSOLIDATED / NON-CONSOLIDATED' : 'consolidated','PERIOD': 'period','PERIOD ENDED' : 'period_ended','RELATING TO' : 'related_qtr','** XBRL' : 'xbrl_url'}
            data = data[cols.keys()]
            data['symbol'] = sym
            data.rename(columns=cols,inplace=True)
            data['period_ended'] = pd.to_datetime(data['period_ended']).dt.date
            data = data[data['period_ended'] > datetime.date(2020,1,1)] 
            data = data[data['consolidated'] == 'Consolidated']
            try:
                data['results'] = data['xbrl_url'].apply(lambda x: parse_xbrl(x))
                data['timestamp'] = timestamp
                data.drop('xbrl_url', axis=1, inplace=True)
                data.to_sql('financial_results',engine,if_exists='append',index=False)
                print(data)#[['consolidated','xbrl_url','results']])
            except:
                pass

In [3]:
# Implementation of general results file having several symbols together

import zipfile,re
import requests
import xml.etree.ElementTree as ET


headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"
}


def parse_xbrl(url):
    result = pd.DataFrame()
    # print(url)
    i = 0
    s = requests.Session()
    response = s.get(url,headers=headers)
    # print(response.content)
    
    # tree = ET.fromstring(response.content)
    
    # tapo = 'C:\\Users\\Santosh Bag\\Downloads\\TAPO.xml'
    # tree = ET.parse(tapo)
    # tree = ET.parse(response.text)
    # root = tree.getroot()
    root = ET.fromstring(response.text)
    results = {}
    for item in root:
        if 'in-bse-fin' in item.tag:
    #         print(item.attrib,item.text)
            results[item.tag.split('}')[1]] = item.text
    finres = pd.DataFrame.from_dict(results, orient='index')
    finres.rename(columns={0:i},inplace=True)
    # print(finres)
    # finres
    # result = pd.concat([result,finres],axis=1)
    # print('RESULT\n',result)
    i = i + 1
    results = json.dumps(results)
    return results

def getSymbol(company_desc):
    print(company_desc)
    sym = company_desc
    try:
        sym = engine.execute("select symbol from security_master where stock_name='"+company_desc+"' limit 1").fetchone()[0]
    except:
        pass
    return sym

eq_dir_qtr = os.path.join('resources','equity_research')
timestamp = datetime.datetime.now()
for r,d,f in os.walk(eq_dir_qtr):
    for file in f:
        if re.search("^FINANCIAL_RESULTS.*csv",file):
        # if re.search("^FINRESULTS_ANNUAL_TATAMOTORS.*csv",file):            
            # sym = os.path.basename(file).split('.')[0].split('_')[2]

            # stockbhavlist.append(os.path.join(r,file))
            file = os.path.join(r,file)
            print(file)
            data = pd.read_csv(file)
            cols = {'COMPANY NAME':'symbol','CONSOLIDATED / NON-CONSOLIDATED' : 'consolidated','PERIOD': 'period','PERIOD ENDED' : 'period_ended','RELATING TO' : 'related_qtr','** XBRL' : 'xbrl_url'}
            data = data[cols.keys()]
            data.rename(columns=cols,inplace=True)
            data['period_ended'] = pd.to_datetime(data['period_ended']).dt.date
            data = data[data['period_ended'] > datetime.date(2020,1,1)] 
            data = data[data['consolidated'] == 'Consolidated']
            try:
                data['results'] = data['xbrl_url'].apply(lambda x: parse_xbrl(x))
                data['timestamp'] = timestamp
                data.drop('xbrl_url', axis=1, inplace=True)
                data['symbol'] = data['symbol'].apply(lambda x: getSymbol(x))                
                # data.to_sql('financial_results',engine,if_exists='append',index=False)
                print(data)#[['consolidated','xbrl_url','results']])
            except:
                pass
            print(data)

resources\equity_research\FINANCIAL_RESULTS.csv
Coforge Limited
Suzlon Energy Limited
Spencer's Retail Limited
Mahindra Logistics Limited
Poly Medicure Limited
ZF Commercial Vehicle Control Systems India Limited
Mangalore Refinery and Petrochemicals Limited
Supreme Industries Limited
Pondy Oxides & Chemicals Limited
Greenlam Industries Limited
IDBI Bank Limited
UCO Bank
Solara Active Pharma Sciences Limited
Digicontent Limited
Dodla Dairy Limited
MMP Industries Limited
Indian Overseas Bank
HDFC Bank Limited
Poonawalla Fincorp Limited
Rossari Biotech Limited
Aarti Surfactants Limited
Gravita India Limited
Arihant Capital Markets Limited
Vikas Lifecare Limited
Shakti Pumps (India) Limited
JK Cement Limited
Kotak Mahindra Bank Limited
Kabra Extrusion Technik Limited
Seshasayee Paper and Boards Limited
Yes Bank Limited
Integra Essentia Limited
Union Bank of India
RBL Bank Limited
Summit Securities Limited
Aurum PropTech Limited
Tree House Education & Accessories Limited
Ksolves India Limit

In [8]:
getSymbol('Gravita India Limited')

Gravita India Limited


'GRAVITA'

In [4]:
def getResults(symbol,period_start=None,period_end=None,period_type='Quarterly'):

    export_dir = os.path.join('downloads')
    end_date = datetime.date.today()
    if period_end is not None:
        end_date = datetime.datetime.strptime(period_end,'%Y-%m-%d')    
    if period_start is not None:
        start_date = datetime.datetime.strptime(period_start,'%Y-%m-%d')
    else:
        start_date = end_date - datetime.timedelta(days=368)

    sql = "select results from financial_results where symbol='{}' and " \
          "period_ended >= '{}' and period_ended <= '{}' and period='{}' and "\
          "timestamp = (select max(timestamp) from financial_results where symbol= '{}') "
    results = pd.read_sql(sql.format(symbol,start_date.strftime('%Y-%m-%d'),end_date.strftime('%Y-%m-%d'),period_type,symbol),engine)
    result = pd.DataFrame()
    for inx in results.index:
        finres = pd.DataFrame.from_dict(json.loads(results.loc[inx]['results']), orient='index')
        finres.rename(columns={0:finres.loc['DateOfEndOfReportingPeriod'][0]},inplace=True)
        result = pd.concat([result,finres],axis=1)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    export_file = os.path.join(export_dir,symbol+"_"+start_date.strftime('%Y%m%d')+"_"+end_date.strftime('%Y%m%d')+"_"+period_type+"_"+timestamp+".csv")
    print(result)
    result.to_csv(export_file)
    return result
    # print(json.loads(results.iloc[1]['results']))
                           

        

In [5]:
getResults('HDFCBANK')

                                                                                           2024-06-30
ScripCode                                                                                      500180
Symbol                                                                                       HDFCBANK
MSEISymbol                                                                                  NOTLISTED
ISIN                                                                                     INE040A01034
NameOfBank                                                                          HDFC Bank Limited
ResultType                                                                             Banking Format
ClassOfSecurity                                                                                Equity
DateOfStartOfFinancialYear                                                                 2024-04-01
DateOfEndOfFinancialYear                                                          

,2024-06-30
ScripCode,500180
Symbol,HDFCBANK
MSEISymbol,NOTLISTED
ISIN,INE040A01034
NameOfBank,HDFC Bank Limited
ResultType,Banking Format
ClassOfSecurity,Equity
DateOfStartOfFinancialYear,2024-04-01
DateOfEndOfFinancialYear,2025-03-31
DateOfBoardMeetingWhenFinancialResultsWereApproved,2024-07-20


In [ ]:
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
assets = pytesseract.image_to_string(Image.open('tapo.png'))
liabilities = pytesseract.image_to_string(Image.open('tapo1.png'))
print(assets)
print(liabilities)

In [ ]:
assets.split('\n')[3]

In [ ]:
key = '1MVD2LBE1UBKFUX0'

from alpha_vantage.fundamentaldata import FundamentalData
import pandas as pd

def get_fundamental_data(symbol):
    api_key = key
    fd = FundamentalData(key=api_key, output_format='pandas')

    # Get company overview
    overview, _ = fd.get_company_overview(symbol)
    
    # Get earnings
    earnings, _ = fd.get_company_earnings(symbol)

    return overview, earnings

if __name__ == "__main__":
    symbol = input("Enter the stock symbol: ").upper()
    try:
        overview, earnings = get_fundamental_data(symbol)
        print(f"Overview for {symbol}:")
        print(overview)
        print("\nEarnings Data for {symbol}:")
        print(earnings)
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
url = "https://www.nseindia.com/get-quotes/equity?symbol=HDFCBANK#Corporate_Actions"

import pandas
import requests



headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"
}

s = requests.Session()
response = s.get(url,headers=headers)
print(response.content)
# data = requests.get



In [ ]:
oc.sort_values(['itm x oi'],ascending=False).head(1)['strike'].values[0]

In [ ]:
import plotly.graph_objects as go
scrips = ['NIFTY 50','NIFTY BANK', 'INDIA VIX']
kitecodemap = pd.read_csv('kitensecodes.csv')
scrips = [kitecodemap[kitecodemap['tradingsymbol'] == scrip]['instrument_token'].values[0] for scrip in scrips]
to_date = datetime.date.today()
from_date = to_date - datetime.timedelta(days=30)
hist_data = []
for token in scrips:
    data = session.historical_data(token,from_date,to_date,interval='day',continuous=False)
    data = pd.DataFrame(data)
#     data['date'] = data['date'].apply(lambda x : datetime.datetime.strptime(str(x), "%Y-%m-%d %H:%M:%S+05:30"))
#     data['date'] = data['date'].apply(lambda x : datetime.datetime.fromtimestamp(x))

    hist_data.append(data[data.columns[:-1]])
# hist_data[0]


In [ ]:
import tkinter as tk

win = tk.Tk()

fig = go.Figure(data=[go.Candlestick(x=hist_data[2]['date'],
                open=hist_data[2]['open'],
                high=hist_data[2]['high'],
                low=hist_data[2]['low'],
                close=hist_data[2]['close'])])
fig.update_layout(xaxis_rangeslider_visible=False)


fig.show()

In [ ]:
# import tradingDB as tdb
# tdbobj = tdb.tradingDB()
# grph = tdbobj.market_snapshot()

# ins = session.instruments(exchange='CDS')
# inspd = pd.DataFrame(ins)
# inspd[['tradingsymbol','instrument_token']].to_csv('kitemcxcodes.csv')

current_month = datetime.date.strftime(datetime.date.today(),'%b')
next_month = datetime.date.strftime(datetime.date.today() + datetime.timedelta(days=31),'%b') 
current_year = datetime.date.strftime(datetime.date.today(),'%y')
crude = 'GOLD'+current_year+next_month.upper()+'FUT'
usdinr = 'USDINR'+current_year+current_month.upper()+'FUT'

ins = session.instruments(exchange='MCX')
inspd = pd.DataFrame(ins)
# inspd[(inspd['tradingsymbol'].str.contains(u'GOLD*****FUT')) & (inspd['segment'] == 'MCX-FUT')].sort_values(by='expiry').head(1)
print(crude+'     '+str(inspd[inspd['tradingsymbol'] == crude]['instrument_token'].values[0]))


# session.quote('MCX:'+crude)

In [ ]:
pd.DataFrame(session.historical_data(65260551,from_date,to_date,'day'))
pd.DataFrame(session.historical_data(2433027,from_date,to_date,'day'))

In [ ]:
pd.DataFrame(session.quote('NSE:NIFTY BANK'))['NSE:NIFTY BANK']

In [ ]:
variety = session.VARIETY_REGULAR
exchange= session.EXCHANGE_NFO
tradingsymbol='TATAPOWER23AUG250CE'
transaction_type=session.TRANSACTION_TYPE_SELL
quantity=3375
product= session.PRODUCT_NRML
order_type = session.ORDER_TYPE_LIMIT
price=15
validity=session.VALIDITY_DAY
disclosed_quantity=None
trigger_price=None
squareoff=None 
stoploss=None
trailing_stoploss=None
tag=None

margin = session.order_margins(variety,exchange, tradingsymbol, transaction_type, quantity, product, order_type, price,
                    validity, disclosed_quantity, trigger_price, squareoff=None, stoploss=None,
                    trailing_stoploss=None, tag=None)

margin

In [ ]:
qt = session.quote('NSE:LIQUIDBEES')
qt

In [ ]:
scrip = 'TATAPOWER'
atm = mu.find_atm(614,5)
atm

In [ ]:
inspd = pd.DataFrame(ins)
#inspd = inspd[(inspd['name'] == scrip) & (inspd['expiry'] == expiry) & (inspd['strike'] >= 18000) & (inspd['strike'] <= 20500) ][['tradingsymbol','strike']]
inspd = inspd[(inspd['name'] == scrip) & (inspd['expiry'] == expiry) & (inspd['strike'] > 0)][['tradingsymbol','strike']]
inspd['tradingsymbol'] = 'NFO:' + inspd['tradingsymbol']
inspd.set_index('tradingsymbol',inplace=True)
inspd
inslist = list(inspd.index)
inslist = inslist[int(len(inslist)/2-len(inslist)/4):int(len(inslist)/2+len(inslist)/4)]
inslist


In [ ]:
def kite_oc(df):
    def kitequote_to_oc(quote):
        oc = pd.DataFrame(quote)
        for key in quote.key:
            qt = quote[key]
            oc = pd.DataFrame(qt)
    for strike in set(list(df['strike'])):
        ce = 'NFO:'+ df[df['strike'] == strike]['tradingsymbol'].head(1).values[0]
        pe = 'NFO:'+ df[df['strike'] == strike]['tradingsymbol'].head(1).values[0]
        

In [ ]:
oc = pd.DataFrame(qt).T
oc.index.name = 'tradingsymbol'
oc1 = oc.merge(inspd,on='tradingsymbol')[['strike','oi','volume','last_price']] 


In [ ]:
scrip = ['TATAPOWER']
expiry_list = [expiry1,expiry2,expiry3]
oc = mu.kite_OC(scrip,expiry_list)
oc.reset_index(inplace=True)

In [ ]:
8 in [1,3,4,5]

In [ ]:
oc.loc[oc['tradingsymbol'].str[-2:] == 'CE'].sort_values(by='oi',ascending=False).head(5).sort_values(by='strike')

In [ ]:
oc.loc[oc['tradingsymbol'].str[-2:] == 'PE'].sort_values(by='oi',ascending=False).head(5).sort_values(by='strike')

In [ ]:
name = 'TATAPOWER'
oi_tree = oc[oc['name'] == name]
for expiry in sorted(set(list(oc['expiry'])),reverse=True):
    oi_tree['oi_ce'+str(expiry)] = oc.loc[(oc['tradingsymbol'].str[-2:] == 'CE') & (oc['expiry'] == expiry),'oi']/oc[(oc['tradingsymbol'].str[-2:] == 'CE') & (oc['expiry'] == expiry)]['oi'].sum()
    oi_tree['oi_pe'+str(expiry)] = -oc.loc[(oc['tradingsymbol'].str[-2:] == 'PE') & (oc['expiry'] == expiry),'oi']/oc[(oc['tradingsymbol'].str[-2:] == 'PE') & (oc['expiry'] == expiry)]['oi'].sum()
oi_tree.fillna(0,inplace=True)

In [ ]:
oi_tree

In [ ]:
import matplotlib.pyplot as plt
oi_tree.sort_values(by=['expiry','strike'],ascending=[True,True],inplace=True)
fig, ax = plt.subplots(1,3,figsize=(20,10),squeeze=False) 

#oi_tree[['oi_ce','oi_pe']].plot(kind='barh')
i= 0
for exp in set(list(oc['expiry'])):
    print(exp)
    ax1 = ax[0,i]
    ax1.set_title(exp)
    ax1.grid()
    ax2 = ax[0,i].twinx()
    x_data = oi_tree[oi_tree['expiry'] == exp]['strike']
    y1_data = oi_tree[oi_tree['expiry'] == exp]['oi_ce'+str(exp)]
    y2_data = oi_tree[oi_tree['expiry'] == exp]['oi_pe'+str(exp)]
    ax1.set_xlabel('OI')
    ax1.set_ylabel('STRIKE')
    ax1.barh(x_data,y1_data,color='b')

    ax2.set_ylabel('STRIKE')
    ax2.barh(x_data,y2_data,color='g')

#     oi_tree[oi_tree['expiry'] == exp].plot(x='strike', y = 'oi_ce'+str(exp), ax = ax[0,i] ,title = exp, kind='barh',ylabel='OI') 
#     oi_tree[oi_tree['expiry'] == exp].plot(x='strike', y = 'oi_pe'+str(exp), ax = ax2,title = exp,secondary_y=True, kind='barh',color='g',ylabel='OI') 
#     ax2.set_ylabel('strike')
    i+=1
plt.show()